In [1]:
import torch
from transformers import AutoTokenizer
from custom_models import load_custom_class
import seaborn as sns
from tokenizers_pos import *
import matplotlib.pyplot as plt

# sub class from modelling_bert "RobertaForSequenceClassification" and override the forward method
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package words to /home/keddie/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# repo_link_nli = "cross-encoder/nli-distilroberta-base"
# repo_link_nli = "sentence-transformers/nli-bert-base"
repo_link_nli = "cross-encoder/nli-MiniLM2-L6-H768"


model_args = {}

tokenizer_nli, model_nli = load_custom_class(repo_link_nli, device, **model_args)

In [3]:
# python eval_scripts.py --path "data/multinli_1.0/multinli_1.0_dev_mismatched.jsonl" --tok_method adj --model roberta --do_custom true
# python eval_scripts.py --path "data/snli_1.0/snli_1.0_test.jsonl" --tok_method adj --model roberta --do_custom true
import json

data = {
    "label": [],
    "sent1": [],
    "sent2": []
}
# data_path = "data/multinli_1.0/multinli_1.0_dev_mismatched.jsonl"
data_path = "data/snli_1.0/snli_1.0_test.jsonl"
with open(data_path, "r") as file:
    for line in file:
        aux_dict = json.loads(line)
        data["label"] += [aux_dict["gold_label"]]
        data["sent1"] += [aux_dict["sentence1"]] # premise
        data["sent2"] += [aux_dict["sentence2"]] # hypothesis

In [4]:
vocabulary_id2tok = {tok_id:tok for tok, tok_id in tokenizer_nli.vocab.items()}
test_nli = [
    ("the cat is large. an ant is small.", "the cat is bigger than the ant.", "entailment"),
    ("dumbo is by the tree and is not small. bambi is an animal that is not large and is hungry.", "dumbo is not smaller than bambi.", "neutral")
    ]

In [5]:
from custom_tokenizer_abstract import CustomTokenizerGeneral
from example_tokenization import custom_tokenization
from tokenizers_pos import *

separator = ""
special_space_token = "Ġ"

custom_tokenizer = CustomTokenizerGeneral(tokenizer_nli, verb_tokenizer, separator=separator, special_space_token=special_space_token)
# custom_tokenizer = CustomTokenizerGeneral(tokenizer_nli, verb_tokenizer, separator="##", special_space_token="")
out = custom_tokenizer((data["sent1"][4], data["sent2"][4]))["input_ids"]

Tensors and operations will be done on cuda:0.


In [11]:
from tqdm import tqdm
limit = 0
custom_tokenizer = CustomTokenizerGeneral(tokenizer_nli, verb_tokenizer, separator=separator, special_space_token=special_space_token)
for p, h in tqdm(zip(data["sent1"][limit:], data["sent2"][limit:]), total=len(data["sent2"][limit:])):
    assert len(custom_tokenizer((p,h), do_lowercase=True)["input_ids"][0]) <= 512, (p,h)
custom_tokenizer = CustomTokenizerGeneral(tokenizer_nli, adjective_tokenizer, separator=separator, special_space_token=special_space_token)
for p, h in tqdm(zip(data["sent1"][limit:], data["sent2"][limit:]), total=len(data["sent2"][limit:])):
    assert len(custom_tokenizer((p,h), do_lowercase=True)["input_ids"][0]) <= 512, (p,h)
custom_tokenizer = CustomTokenizerGeneral(tokenizer_nli, noun_tokenizer, separator=separator, special_space_token=special_space_token)
for p, h in tqdm(zip(data["sent1"][limit:], data["sent2"][limit:]), total=len(data["sent2"][limit:])):
    assert len(custom_tokenizer((p,h), do_lowercase=True)["input_ids"][0]) <= 512, (p,h)
custom_tokenizer = CustomTokenizerGeneral(tokenizer_nli, custom_tokenization, separator=separator, special_space_token=special_space_token)
for p, h in tqdm(zip(data["sent1"][limit:], data["sent2"][limit:]), total=len(data["sent2"][limit:])):
    assert len(custom_tokenizer((p,h), do_lowercase=True)["input_ids"][0]) <= 512, (p,h, len(custom_tokenizer((p,h), do_lowercase=True)["input_ids"][0]))

Tensors and operations will be done on cuda:0.


100%|██████████| 10000/10000 [01:15<00:00, 133.26it/s]


Tensors and operations will be done on cuda:0.


100%|██████████| 10000/10000 [01:14<00:00, 134.46it/s]


Tensors and operations will be done on cuda:0.


100%|██████████| 10000/10000 [01:13<00:00, 135.65it/s]


Tensors and operations will be done on cuda:0.


100%|██████████| 10000/10000 [00:01<00:00, 6700.73it/s]


In [7]:
from collections import defaultdict

def run_single_experiment(premise_hypothesis_test):
    prob_list = defaultdict(list)
    granular_pred = get_prediction(premise_hypothesis_test, model_nli, custom_tokenizer)
    for label, prob in granular_pred["all_probs"].items():
        prob_list[label] += [prob]
    prob_list["pred_label"] += [granular_pred["label"]]

    return prob_list

results = []
for idx, problem in enumerate(test_nli):
    results += [(run_single_experiment(problem[:2]), problem[-1])]

[101, 1996, 4937, 2003, 2312, 1012, 2019, 14405, 2003, 2235, 1012, 102, 1996, 4937, 2003, 7046, 2084, 1996, 14405, 1012, 102]
[101, 12873, 2080, 2003, 2011, 1996, 3392, 1998, 2003, 2025, 2235, 1012, 25307, 5638, 2003, 2019, 4111, 2008, 2003, 2025, 2312, 1998, 2003, 7501, 1012, 102, 12873, 2080, 2003, 2025, 3760, 2084, 25307, 5638, 1012, 102]


/home/keddie/Desktop/2024-2025/LoLa/Project/custom_tokenizer_abstract.py:69: UserWarning: Warning: 'dumbo' was not found in the original tokenizer's vocabulary; 'dumbo' will be split into '['dumb', '##o']' using the original tokenizer.
  aux_token_list += self.get_token_id(token)
/home/keddie/Desktop/2024-2025/LoLa/Project/custom_tokenizer_abstract.py:69: UserWarning: Warning: ' by' was not found in the original tokenizer's vocabulary; ' by' will be split into '['by']' using the original tokenizer.
  aux_token_list += self.get_token_id(token)
/home/keddie/Desktop/2024-2025/LoLa/Project/custom_tokenizer_abstract.py:69: UserWarning: Warning: ' tree' was not found in the original tokenizer's vocabulary; ' tree' will be split into '['tree']' using the original tokenizer.
  aux_token_list += self.get_token_id(token)
/home/keddie/Desktop/2024-2025/LoLa/Project/custom_tokenizer_abstract.py:69: UserWarning: Warning: ' and' was not found in the original tokenizer's vocabulary; ' and' will be sp

In [8]:
results

[(defaultdict(list,
              {'LABEL_1': [0.5144256949424744],
               'LABEL_0': [0.485574334859848],
               'pred_label': ['LABEL_1']}),
  'entailment'),
 (defaultdict(list,
              {'LABEL_0': [0.5117776989936829],
               'LABEL_1': [0.48822230100631714],
               'pred_label': ['LABEL_0']}),
  'neutral')]